# import the models from the config


In [1]:
import yaml
import importlib
import os
import joblib
import pickle
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
import os
import pickle
from sklearn.metrics import f1_score
import copy
import pandas as pd
import plotly.express as px

import warnings

IGNORE_WARNINGS = True


if IGNORE_WARNINGS:
    warnings.filterwarnings("ignore")





# Set the maximum width of the display for strings
pd.set_option('display.max_colwidth', 30)



def load_models_from_config(config_file):
    with open(config_file, 'r') as file:
        config = yaml.safe_load(file)
    
    models = {}
    for model_name, model_config in config['models'].items():
        steps = []
        for step_config in model_config['steps']:
            module = importlib.import_module(step_config['module'])
            class_ = getattr(module, step_config['class'])
            if 'strategy' in step_config:
                step = (step_config['name'], class_(strategy=step_config['strategy']))
            else:
                step = (step_config['name'], class_())
            steps.append(step)

        # Add GridSearchCV if present in the config
        if 'grid_search' in model_config:
            param_grid = model_config['grid_search']['param_grid']
            cv = model_config['grid_search']['cv']
            scoring = model_config['grid_search']['scoring']
            grid_search = GridSearchCV(Pipeline(steps), param_grid, cv=cv, scoring=scoring)
            models[model_name] = grid_search
        else:
            models[model_name] = Pipeline(steps)
    
    
    return models

# Load models from YAML configuration
models = load_models_from_config('models_config.yaml')


# Train the models

In [2]:

# Path to the train data folder
train_data_folder = 'data/train'

# Create a dictionary to store trained models
trained_models = {}

# Iterate over files in train data folder
for filename in os.listdir(train_data_folder):
    file_path = os.path.join(train_data_folder, filename)
    
    # Load data from .pkl file
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        X_train, y_train = data.drop(columns='classes'), data['classes'].astype("category")
    
    # Iterate over models
    for model_name, model_pipeline in models.items():
        # Create a copy of the model pipeline
        model = copy.deepcopy(model_pipeline)
        
        # Train model
        model.fit(X_train, y_train)
        
        # Save trained model in dictionary with keys based on filename and model name
        key = f'{filename}_{model_name}'
        trained_models[key] = model
    


In [3]:
# Path to the testing data folder
test_data_folder = 'data/test'

# Dictionary to store F1 micro scores for each model
f1_scores = {}

res = []

# Iterate over files in test data folder
for filename in os.listdir(test_data_folder):
    file_path = os.path.join(test_data_folder, filename)
    
    # Load data from .pkl file
    with open(file_path, 'rb') as f:
        data = pickle.load(f)
        X_test, y_test = data.drop(columns='classes'), data['classes'].astype("category")
    
    # Extract model name from the filename
    model_name, _ = filename.split('.pkl')
    model_name, _ = (model_name + '|').split('|')
    
    # Get the trained model corresponding to the model name
    model_keys = [key for key in trained_models if key.startswith(model_name)]

    for model_key in model_keys:

        model = trained_models.get(model_key)
        
        # Predict labels using the model
        y_pred = model.predict(X_test)
        
        # Calculate F1 micro score
        f1_micro = f1_score(y_test, y_pred, average='micro')
        
        res.append([model_key, f1_micro])



In [4]:
df = pd.DataFrame(res,columns=['name', 'f1_micro'])


df[['dataset','model']] = df['name'].str.split('.pkl_', expand=True)
df[['dataset','pollution']] = df['dataset'].str.split('|', expand=True)

df = df.fillna(0)

In [5]:
def sort_df(df):
    return df.sort_values(by=['accuracy', 'completeness', 'dupp_fac'], ascending=True, na_position='first')


In [6]:
# Extracting coefficients into separate columns
df['completeness'] = df['pollution'].str.extract(r'completness_pollution_([0-9.]+)')
df['accuracy'] = df['pollution'].str.extract(r'feature_accuracy_pollution_([0-9.]+)')
df['dupp_fac'] = df['pollution'].str.extract(r'duplicate_factor_([0-9]+)')

# Converting columns to appropriate data types
df['completeness'] = df['completeness'].astype(float)
df['accuracy'] = df['accuracy'].astype(float)
df['dupp_fac'] = df['dupp_fac'].astype(float)


# Ordering the DataFrame based on the columns
df =  sort_df(df)

In [7]:
# Apply group by and assign an index representing the order within each group
df['group_index'] = df.groupby(['dataset', 'model']).cumcount()

# Apply diff within each group
df['diff'] = df.groupby(['dataset', 'model'])['f1_micro'].diff()

pol_col = df[['completeness', 'accuracy', 'dupp_fac']].columns
df_col = df.columns

In [8]:
li = df.pollution.unique()
agg = df.groupby(['model','group_index'])['diff'].mean()
agg = pd.DataFrame(agg).reset_index()

agg['pollution'] = agg['group_index'].apply(lambda x: li[x])

agg['completeness'] = agg['pollution'].str.extract(r'completness_pollution_([0-9.]+)')
agg['accuracy'] = agg['pollution'].str.extract(r'feature_accuracy_pollution_([0-9.]+)')
agg['dupp_fac'] = agg['pollution'].str.extract(r'duplicate_factor_([0-9]+)')


fig = px.line(agg.fillna(0), x="pollution", y="diff", color='model')

fig.update_xaxes(showticklabels=False)

fig.show()

In [9]:
fig = px.line(agg.dropna(subset=pol_col), x="pollution", y="diff", color='model')

fig.update_xaxes(showticklabels=False)

fig.show()

In [10]:
viz = agg[agg[pol_col.difference(['dupp_fac'])].isnull().all(axis=1) | agg[pol_col].isnull().all(axis=1)]

fig = px.line(viz.fillna(0), x="pollution", y="diff", color='model')

fig.update_xaxes(showticklabels=False)

fig.show()

In [11]:
viz = agg[agg[pol_col.difference(['accuracy'])].isnull().all(axis=1) | agg[pol_col].isnull().all(axis=1)]

fig = px.line(viz.fillna(0), x="pollution", y="diff", color='model')

fig.update_xaxes(showticklabels=False)

fig.show()

In [12]:
viz = agg[agg[pol_col.difference(['completeness'])].isnull().all(axis=1) | agg[pol_col].isnull().all(axis=1)]

fig = px.line(viz.fillna(0), x="pollution", y="diff", color='model')

fig.update_xaxes(showticklabels=False)

fig.show()

In [13]:
groups = df.groupby('dataset')

moving_diff = pd.DataFrame()


for dataset, param in groups:
    
    param = sort_df(param)
    
    fig = px.line(param, x="pollution", y="f1_micro", color='model', title=param.dataset.iloc[0] )

    fig.update_xaxes(showticklabels=False)

    fig.show()
                       

In [14]:
pol_col = df[['completeness', 'accuracy', 'dupp_fac']].columns
df_col = df.columns


groups =df[df[pol_col.difference(['completeness'])].isnull().all(axis=1) | df[pol_col].isnull().all(axis=1)].groupby('dataset')

for dataset, param in groups:
    
    param = sort_df(param)
    
    fig = px.line(param, x="pollution", y="f1_micro", color='model', title=param.dataset.iloc[0] )

    fig.update_xaxes(showticklabels=False)

    fig.show()

In [15]:
pol_col = df[['completeness', 'accuracy', 'dupp_fac']].columns
df_col = df.columns


groups =df[df[pol_col.difference(['accuracy'])].isnull().all(axis=1) | df[pol_col].isnull().all(axis=1)].groupby('dataset')

for dataset, param in groups:
    
    param = sort_df(param)
    
    fig = px.line(param, x="pollution", y="f1_micro", color='model', title=param.dataset.iloc[0] )

    fig.update_xaxes(showticklabels=False)

    fig.show()

In [16]:
pol_col = df[['completeness', 'accuracy', 'dupp_fac']].columns
df_col = df.columns


groups =df[df[pol_col.difference(['dupp_fac'])].isnull().all(axis=1) | df[pol_col].isnull().all(axis=1)].groupby('dataset')

for dataset, param in groups:
    
    param = sort_df(param)
    
    fig = px.line(param, x="pollution", y="f1_micro", color='model', title=param.dataset.iloc[0] )

    fig.update_xaxes(showticklabels=False)

    fig.show()